In [1]:
import pandas as pd
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('flood.csv')


In [3]:
df.head()

,Unnamed: 0,tweet_text,tokenized_text,Subjectivity,polarity,nb_pronouns,nb_numerals,nb_exclm,nb_ques,count_modal_verbs,count_wh_words,nb_slang,label
0,0,RT @lilid426: Everyone has all these intense r...,"['RT', '@', 'lilid426', ':', 'Everyone', 'inte...",1.000000,0.200000,1,0,0,0,0,0,0,not_related_or_irrelevant
1,1,@kkfred12 @StayDvpper oh yea I would like to t...,"['@', 'kkfred12', '@', 'StayDvpper', 'oh', 'ye...",0.000000,0.000000,3,0,0,0,1,0,0,not_related_or_irrelevant
2,2,"RT @fruitpopniall: ""1D are about to split up"" ...","['RT', '@', 'fruitpopniall', ':', '``', '1D', ...",0.000000,0.000000,0,0,0,0,0,0,0,not_related_or_irrelevant
3,3,@Harry_Styles You are my ☀ You 💡 up my who...,"['@', 'Harry_Styles', 'You', '☀', 'You', '💡', ...",0.408333,0.133333,9,0,0,0,0,0,0,not_related_or_irrelevant
4,4,RT @abonetti420: Swinging blunts round sun up ...,"['RT', '@', 'abonetti420', ':', 'Swinging', 'b...",0.344444,-0.177778,0,0,0,0,0,0,0,not_related_or_irrelevant


In [6]:
dff = df.sample(frac=1)
df_train = dff[:1000]
df_test = dff[-1000:]
print(df_train.shape)
print(df_test.shape)
X_train, y_train = df_train['tweet_text'].values, df_train['label'].values
X_test, y_test = df_test['tweet_text'].values, df_test['label'].values

(1000, 13)
(1000, 13)


In [11]:
#check distribution
import numpy as np
unique, counts = np.unique(y_train, return_counts=True)
print(np.asarray((unique, counts)))
unique, counts = np.unique(y_test, return_counts=True)
print(np.asarray((unique, counts)))

[['caution_and_advice' 'displaced_people_and_evacuations'
  'donation_needs_or_offers_or_volunteering_services'
  'infrastructure_and_utilities_damage' 'injured_or_dead_people'
  'missing_trapped_or_found_people' 'not_related_or_irrelevant'
  'other_useful_information' 'sympathy_and_emotional_support']
 [19 38 140 45 287 37 159 234 41]]
[['caution_and_advice' 'displaced_people_and_evacuations'
  'donation_needs_or_offers_or_volunteering_services'
  'infrastructure_and_utilities_damage' 'injured_or_dead_people'
  'missing_trapped_or_found_people' 'not_related_or_irrelevant'
  'other_useful_information' 'sympathy_and_emotional_support']
 [26 26 138 51 333 32 141 214 39]]


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
unigram_count_vectorizer = CountVectorizer(encoding='latin-1', binary=False, min_df=2, stop_words='english')
X_train_vec = unigram_count_vectorizer.fit_transform(X_train)
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=1, max_iter=2000)
svm_clf.fit(X_train_vec,y_train)
feature_ranks = sorted(zip(svm_clf.coef_[0], unigram_count_vectorizer.get_feature_names()))
top_10 = feature_ranks[-10:]
print("Top 10 words")
for i in range(0, len(top_10)):
    print(top_10[i])
print()

Top 10 words
(0.6356465451205757, 'river')
(0.6609234851089985, 'havoc')
(0.6627512865202669, 'instructions')
(0.7013598727733956, 'indianews')
(0.7439142363082544, 'hungary')
(0.7439142363082544, 'poland')
(0.7479769529370441, 'save')
(0.8254167374555, 'worse')
(0.9099215116924175, 'emergency')
(1.1062843758982894, 'followback')



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
X_test_vec = unigram_count_vectorizer.transform(X_test)
y_pred = svm_clf.predict(X_test_vec)
svm_clf.score(X_test_vec,y_test)


0.679

In [14]:
from sklearn.metrics import confusion_matrix
y_pred = svm_clf.predict(X_test_vec)
cm=confusion_matrix(y_test, y_pred)
print(cm)
print()

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


[[  6   0   2   2   2   0   1  13   0]
 [  1   6   3   0   3   4   1   7   1]
 [  1   2  82   4   5   6   4  32   2]
 [  3   2   2  12   7   2   3  19   1]
 [  1   2   2   0 300   2   4  22   0]
 [  1   4   2   0   2  12   1  10   0]
 [  0   1   2   0   4   0 124   8   2]
 [  6   4  37  10  11   3  11 127   5]
 [  1   1   6   0   1   0   3  17  10]]

                                                   precision    recall  f1-score   support

                               caution_and_advice       0.30      0.23      0.26        26
                 displaced_people_and_evacuations       0.27      0.23      0.25        26
donation_needs_or_offers_or_volunteering_services       0.59      0.59      0.59       138
              infrastructure_and_utilities_damage       0.43      0.24      0.30        51
                           injured_or_dead_people       0.90      0.90      0.90       333
                  missing_trapped_or_found_people       0.41      0.38      0.39        32
         

In [16]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .


Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 13.95 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 132 kB 24.0 MB/s 
     |████████████████████████████████| 9.1 MB 49.8 MB/s 
     |████████████████████████████████| 79 kB 9.7 MB/s 
     |████████████████████████████████| 139 kB 74.5 MB/s 

In [17]:
from bert_sklearn import BertClassifier
model = BertClassifier()         # text/text pair classification
print(model)
model.fit(X_train, y_train)


Building sklearn text classifier...
BertClassifier()


100%|██████████| 231508/231508 [00:00<00:00, 21577630.06B/s]


Loading bert-base-uncased model...


100%|██████████| 433/433 [00:00<00:00, 223496.63B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 900, validation data size: 100



...


Epoch 2, Train loss: 1.1081, Val loss: 1.0450, Val accy: 71.00%



Training  :   0%|          | 0/29 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.8354, Val loss: 1.0018, Val accy: 68.00%



BertClassifier(do_lower_case=True,
               label_list=array(['caution_and_advice', 'displaced_people_and_evacuations',
       'donation_needs_or_offers_or_volunteering_services',
       'infrastructure_and_utilities_damage', 'injured_or_dead_people',
       'missing_trapped_or_found_people', 'not_related_or_irrelevant',
       'other_useful_information', 'sympathy_and_emotional_support'],
      dtype=object))

In [18]:
model.score(X_test, y_test)


Testing:   0%|          | 0/125 [00:00<?, ?it/s]


Loss: 0.9012, Accuracy: 71.10%


71.1